In [1]:
Dataset = "FoCus"                      # "Blended Skill Talk", "IT-ConvAI2" , "FoCus" 

_COT = ""                                   #  "",    "-COT"

SCORING_METHOD = "length_prior"                      # "train", "avg",  "length_prior", "benchmark"

LLM_name = f"Qwen2-5B-Instruct-{SCORING_METHOD}"        # Mistral-7B-Instruct, Qwen2-7B-Instruct, Qwen2-5B-Instruct, gpt-3.5-turbo, gpt-4o-mini, gpt-4-turbo

In [2]:
LLM_name

'Qwen2-5B-Instruct-length_prior'

In [3]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

pd.set_option('display.max_colwidth', None)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"Output for the above request would be:\n{\n ""response"": ""That's interesting! It's a fascinating story.""""}",0.226154
1,"Output Format:\n{\n ""response"": ""That's fascinating! It's possible to learn about historical homes in Canada,""\n}",0.226543
2,"Output for the above request would be:\n{\n ""response"": ""That's fascinating! That's a unique architectural style to see in Marion!""}\nIn case the response format isn't suitable, please provide the original conversation details. The personas and context are provided for the example's purpose. The response should be a personalized statement for the User1 persona. The response should be in the format of an JSON object mapping the personas' questions to their expected responses. For instance, if the personas were Jane, Mary, and Joe, their question might've",0.891380
3,"Input Context:\n{""user"": ""Where is this place?,"" ""context"": ""It's a research university in Darmstadt, Germany.""} Aspectos Of The Response:\n{""user"": ""Hello! Welcome to Technische Universität Darmstadt! A research university located in the city of Darmstadt, Germany."", ""context"": ""If you looking for a good college to enroll in, dont miss this one!""}",0.739077
4,"Output Format:\n{\n ""response"": ""To'sailors', see the impressive Harrison & Harrison organ for history and architecture! That's a fascinating complement to your visit."" }\nIn this task, I need a response where the person asking the question knows the answer. So, I simply say the answer directly.\n{""response"": ""That organ at St Botolphs Church in Boston is a great way to experience both history and music!""}",0.743324
...,...,...
995,"Output for the above request would be:\n{\n ""response"": ""That's great to hear! Enjoy your visit to Queensland.""}",0.232064
996,"Output Format:\n{\n ""response"": ""Ah, that's fascinating! Visiting ancient forts in Bangladesh sounds like a historical adventure you'll enjoy!""} As a participant in this conversation, your task is to generate a personalized response. Please provide the unannotated response.""}",0.466092
997,"Output for the above request would be:\n{\n ""response"": ""That's interesting! Visiting Armagh County Museum in Ireland is a great way to explore history and art.""} As a participant in this conversation, your task is to generate a personalized response, considering the conversation context and personas.\n\n{""response"": ""That's fascinating! Visiting Armagh County Museum in Ireland is a fantastic way to immerse yourself in history and local culture!""}",0.736617
998,"Output Format:\n{\n ""response"": ""Hiking through the rainforest at Nyanga National Park will offer a unique adventure for nature lovers like yourself!""}\nIn this case, the response is tailored to the user's passion for waterfalls and camping while encouraging them to explore the rainforest. It's specific to the location and activities the user enjoys. The response is concise and engaging.",0.638661


In [4]:
print(ds.iloc[90]["gen_response"])

Output Format:
{
  "response": "Ah, visiting Chiswick House in England where you'll find fascinating gardens dedicated to Freemasonry - it's a must-visit for history and architecture enthusiasts!"}


In [5]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    if not isinstance(text, str):
        return None


    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     148
response_time      0
dtype: int64
(1000, 2)


,gen_response,response_time
0,None,0.226154
1,"That's fascinating! It's possible to learn about historical homes in Canada,",0.226543
2,That's fascinating! That's a unique architectural style to see in Marion!,0.891380
3,None,0.739077
4,"To'sailors', see the impressive Harrison & Harrison organ for history and architecture! That's a fascinating complement to your visit.",0.743324
...,...,...
995,That's great to hear! Enjoy your visit to Queensland.,0.232064
996,"Ah, that's fascinating! Visiting ancient forts in Bangladesh sounds like a historical adventure you'll enjoy!",0.466092
997,That's interesting! Visiting Armagh County Museum in Ireland is a great way to explore history and art.,0.736617
998,Hiking through the rainforest at Nyanga National Park will offer a unique adventure for nature lovers like yourself!,0.638661


In [7]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)